In [1]:
import findspark

findspark.add_jars('/app/postgresql-42.1.4.jar')
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Stocks:ETL").getOrCreate()

In [3]:
spark.version

'2.4.5'

In [4]:
stocks_dir = '/dataset/stocks-small'

In [11]:
import sys

from pyspark.sql import SparkSession

# UDF
from pyspark.sql.types import StringType
#
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [13]:
df = spark.read \
    .option("header", True) \
    .option("inferSchema", True) \
    .csv(stocks_dir)

In [14]:
# df.count()
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- OpenInt: integer (nullable = true)



In [15]:
df.show()

+-------------------+------+------+------+------+------+-------+
|               Date|  Open|  High|   Low| Close|Volume|OpenInt|
+-------------------+------+------+------+------+------+-------+
|1962-01-02 00:00:00| 6.413| 6.413|6.3378|6.3378|467056|      0|
|1962-01-03 00:00:00|6.3378|6.3963|6.3378|6.3963|350294|      0|
|1962-01-04 00:00:00|6.3963|6.3963|6.3295|6.3295|314365|      0|
|1962-01-05 00:00:00|6.3211|6.3211|6.1958|6.2041|440112|      0|
|1962-01-08 00:00:00|6.2041|6.2041|6.0373| 6.087|655676|      0|
|1962-01-09 00:00:00|6.1208|6.2376|6.1208|6.1621|592806|      0|
|1962-01-10 00:00:00|6.1707|6.2041|6.1707|6.1707|359274|      0|
|1962-01-11 00:00:00|6.1875|6.2376|6.1875|6.2376|386220|      0|
|1962-01-12 00:00:00|6.2543|6.2962|6.2543|6.2543|529933|      0|
|1962-01-15 00:00:00|6.2708|6.2962|6.2708|6.2792|305383|      0|
|1962-01-16 00:00:00|6.2708|6.2708|6.2128|6.2128|305383|      0|
|1962-01-17 00:00:00|6.1875|6.1875|6.0956|6.1125|502984|      0|
|1962-01-18 00:00:00|6.12

In [17]:
df = df.withColumn('filename', F.input_file_name())

In [22]:
df.show(truncate=False)

+-------------------+------+------+------+------+------+-------+---------------------------------------+
|Date               |Open  |High  |Low   |Close |Volume|OpenInt|filename                               |
+-------------------+------+------+------+------+------+-------+---------------------------------------+
|1962-01-02 00:00:00|6.413 |6.413 |6.3378|6.3378|467056|0      |file:///dataset/stocks-small/ibm.us.txt|
|1962-01-03 00:00:00|6.3378|6.3963|6.3378|6.3963|350294|0      |file:///dataset/stocks-small/ibm.us.txt|
|1962-01-04 00:00:00|6.3963|6.3963|6.3295|6.3295|314365|0      |file:///dataset/stocks-small/ibm.us.txt|
|1962-01-05 00:00:00|6.3211|6.3211|6.1958|6.2041|440112|0      |file:///dataset/stocks-small/ibm.us.txt|
|1962-01-08 00:00:00|6.2041|6.2041|6.0373|6.087 |655676|0      |file:///dataset/stocks-small/ibm.us.txt|
|1962-01-09 00:00:00|6.1208|6.2376|6.1208|6.1621|592806|0      |file:///dataset/stocks-small/ibm.us.txt|
|1962-01-10 00:00:00|6.1707|6.2041|6.1707|6.1707|359274

In [23]:
df_lookup = spark.read.csv('/dataset/yahoo-symbols-201709.csv')

In [24]:
df_lookup.show()

+------+--------------------+--------+--------------------+-------+
|   _c0|                 _c1|     _c2|                 _c3|    _c4|
+------+--------------------+--------+--------------------+-------+
|Ticker|                Name|Exchange|       Category Name|Country|
|  OEDV|Osage Exploration...|     PNK|                null|    USA|
|  AAPL|          Apple Inc.|     NMS|Electronic Equipment|    USA|
|   BAC|Bank of America C...|     NYQ|  Money Center Banks|    USA|
|  AMZN|    Amazon.com, Inc.|     NMS|Catalog & Mail Or...|    USA|
|     T|           AT&T Inc.|     NYQ|Telecom Services ...|    USA|
|  GOOG|       Alphabet Inc.|     NMS|Internet Informat...|    USA|
|    MO|  Altria Group, Inc.|     NYQ|          Cigarettes|    USA|
|   DAL|Delta Air Lines, ...|     NYQ|      Major Airlines|    USA|
|    AA|   Alcoa Corporation|     NYQ|            Aluminum|    USA|
|   AXP|American Express ...|     NYQ|     Credit Services|    USA|
|    DD|E. I. du Pont de ...|     NYQ|Agricultur

In [25]:
filename = 'file:///dataset/stocks-small/ibm.us.txt'
# => IBM